In [117]:
import time
import requests

import numpy as np
import pandas as pd 

from tqdm import tqdm

In [1]:
l = []
for i, mask in pivot.loc['2022-04-16'].isna().items():
    if mask == True:
        if np.isnan(pivot.loc['2022-04-17', i]) == False:
            l.append(i)
            
len(l)

NameError: name 'pivot' is not defined

In [97]:
def chunked_strings(input_list, chunk_size=5):
    return [",".join(input_list[i:i+chunk_size]) for i in range(0, len(input_list), chunk_size)]

coin_lst = chunked_strings(l)

In [119]:
api_key = "f5c86f58-43ef-4ee9-8f75-b5cca80369c7"
url = "https://pro-api.coinmarketcap.com/v2/cryptocurrency/ohlcv/historical"
headers = {'X-CMC_PRO_API_KEY': 'f5c86f58-43ef-4ee9-8f75-b5cca80369c7'}

final_df = pd.DataFrame()
error_list = []

for c_lst in tqdm(coin_lst):
    params = {'id':c_lst,
              'count':10000,
              'time_start':'2021-08-27',
              'time_end':'2022-04-16'
              }
    
    req = requests.get(url=url,
                       params=params,
                       headers=headers)
    
    for key, value in req.json()['data'].items():
        try:
            df = pd.DataFrame([i['quote']['USD'] for i in value['quotes']])
            df.loc[:, 'coin_id'] = str(value['id'])
            final_df = pd.concat([final_df, df])
        except:
            error_list.append({key:value})
    
    time.sleep(2)
    final_df.to_pickle('cmc_add.pickle')

100%|██████████| 489/489 [25:44<00:00,  3.16s/it]
